In [1]:
import pandas as pd

# 载入数据
data = pd.read_csv('./data_all.csv')

# 去除重复记录
data.drop_duplicates(inplace=True)

# 处理缺失值，这里以简单填充为例，实际操作应根据具体情况定制
data.fillna('未知', inplace=True)

# 显示处理后的数据头部信息以确认清洗效果
print(data.head())


   id         company           title    salary education  \
0   0      广东倾云科技有限公司  【初级】web前端开发工程师  2-4K·13薪        大专   
1   1    火眼科技（天津）有限公司         IT运维工程师      2-4K        大专   
2   2  郑州玉带信息技术有限责任公司    实习web前端开发工程师      1-6K        大专   
3   3      广东天勤科技有限公司     兴宁前端、程序员实习生      1-4K        大专   
4   4    武汉赢月网络科技有限公司           运维工程师      2-3K        大专   

                                         description hiring_manager  \
0  1.熟悉HTML5、JavaScript、CSS3;2.熟悉使用css3的flex/grid...            梁先生   
1  一、工作内容1、软件数据分析，月/季报告撰写2、软件平台学习，能够熟练讲解，在使用过程中能够...            刘向宇   
2  参加公司自研产品的Web和微信端小程序开发，使用vue、elementUI和eCharts，...            郭先生   
3  实习生要求：1、需要在梅州兴宁市长期工作（家在梅州市内）2、统招全日制大专或以上学历的应届毕...            林先生   
4                                    负责日常网络及各子系统管理维护             赵爽   

  last_active             address  \
0       2周内活跃      中山广东倾云科技有限公司一层   
1        刚刚活跃       衡水桃城区众成大厦2407   
2        刚刚活跃    郑州二七区升龙世纪花园壹区1号楼   
3          在线        梅州兴宁市天勤网络37号   
4        刚刚活跃  武汉洪山区长

In [2]:
unique_educations = data['education'].unique()
print(unique_educations)


['大专' '本科' '硕士' '博士']


In [4]:
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer

# 配置jieba分词
jieba.initialize()

# 加载一个更全面的停用词列表，包括中文和英文的常见停用词
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    stop_words = set([line.strip() for line in f.readlines()])

# 分句函数，中文文本中主要以句号、逗号、分号、感叹号等为分隔符
def split_sentences(text):
    separators = ['。', '；', '！', '\n', '，', '.','/']
    sentences = []
    sentence = ''
    for char in text:
        sentence += char
        if char in separators:
            sentences.append(sentence.strip())
            sentence = ''
    if sentence:
        sentences.append(sentence.strip())
    return sentences

# 分词和清洗函数
def tokenize_and_clean(text):
    words = jieba.cut(text, cut_all=False)
    cleaned_words = [word for word in words if word.lower() not in stop_words and len(word) > 1]  # 删除停用词和单字
    return ' '.join(cleaned_words)

# 假设我们已经加载了DataFrame 'data'，且其中包含一个名为'description'的列，存放职位描述
descriptions = data['description'].apply(lambda x: ' '.join([tokenize_and_clean(sentence) for sentence in split_sentences(x)]))

# 初始化TF-IDF向量化器，同时使用一个正则表达式作为token_pattern来更好地处理中英文混合文本
vectorizer = TfidfVectorizer(max_features=1000, token_pattern=r'(?u)\b\w+\b')

# 对清洗后的职位描述进行TF-IDF处理
tfidf_matrix = vectorizer.fit_transform(descriptions)
feature_names = vectorizer.get_feature_names_out()
print(feature_names)
# 接下来，可以使用tfidf_matrix进行聚类分析等后续处理



['00' '10' '11' '12' '13' '14' '15' '17' '18' '20' '30' '35' '3d' 'ai'
 'ajax' 'android' 'angular' 'api' 'app' 'arm' 'autosar' 'bgp' 'bim'
 'bootstrap' 'bug' 'c' 'cad' 'can' 'ccie' 'ccnp' 'cisco' 'css' 'css3'
 'c语言' 'design' 'development' 'div' 'dsp' 'eda' 'element' 'erp' 'es6'
 'excel' 'experience' 'fpga' 'git' 'go' 'gulp' 'h3c' 'h5' 'hcie' 'html'
 'html5' 'http' 'i2c' 'ios' 'ip' 'is' 'it' 'java' 'javascript' 'jquery'
 'js' 'json' 'linux' 'matlab' 'mcu' 'mvc' 'mysql' 'net' 'node' 'nodejs'
 'office' 'opencv' 'oracle' 'ospf' 'pc' 'pcb' 'php' 'plc' 'ppt' 'ps'
 'python' 'pytorch' 'qt' 'react' 'redis' 'server' 'shell' 'soc' 'socket'
 'software' 'spi' 'spring' 'sql' 'sqlserver' 'svn' 'tcp' 'team'
 'tensorflow' 'typescript' 'uart' 'ui' 'uni' 'uniapp' 'vpn' 'vue' 'w3c'
 'web' 'webpack' 'windows' 'word' 'work' 'you' '一个' '一体化' '一定' '一年' '一种'
 '一起' '一金' '一门' '三年' '三维' '上位' '上海' '上班时间' '上级' '上级领导' '上线' '上进心' '不同'
 '不定期' '不断' '专业' '专业本科' '专业知识' '专利' '专家' '专科' '业务' '业界' '业绩' '两年' '严谨'
 '个人' '中心' '

In [5]:
print(descriptions[:10])

0     熟悉 HTML5 JavaScript CSS3 熟悉 使用 css3 flex grid...
1    工作 内容 软件 数据分析  报告 撰写 软件平台 学习 能够 熟练 讲解 使用 过程 能够...
2    参加 公司 自研 产品 Web 微信端 程序开发 使用 vue elementUI eCha...
3    实习生 要求 需要 梅州 兴宁市 长期 工作 梅州市 统招 全日制 大专 以上学历 应届 毕...
4                                   负责 日常 网络 子系统 管理 维护
5     专科 以上学历 计算机 通信 电子 相关 专业  热爱 软件开发 工作 移动 产品 浓厚兴...
6    职位 描述 负责 公司 项目 网页 前端 设计 制作 维护 网页 互动 服务 产品 研发 网...
7    任职 要求 精通 HTML CSS JavaScript 良好 编码 习惯 代码 洁癖 优先...
8    岗位职责 负责 公司 相关 web 产品 前端开发 岗位 要求 工作 经验 要求 需要 编码...
9    培养 新人 愿意 学习 勤奋 刻苦 大专 以上学历 游戏 开发 编程 行业 热爱 游戏 开发 大佬
Name: description, dtype: object


In [6]:
from sklearn.cluster import KMeans

# 设定聚类数为 5，这个数字可以根据实际情况调整
n_clusters = 8

# 初始化 KMeans
kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# 对TF-IDF矩阵进行聚类
kmeans.fit(tfidf_matrix)

# 获取每个职位描述所属的聚类标签
cluster_labels = kmeans.labels_

# 将聚类标签添加到原始数据中，以便分析
data['cluster'] = cluster_labels

# 查看每个聚类的大小
cluster_sizes = data['cluster'].value_counts()

# 打印结果
print(cluster_sizes)


D:\anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


1    2192
4    1615
6    1223
0    1165
7    1037
5     819
3     442
2     269
Name: cluster, dtype: int64


In [7]:
from sklearn.cluster import KMeans

# 设定聚类数目为 8，这个数字可以根据实际情况调整
n_clusters = 8

# 初始化 KMeans
kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# 对 TF-IDF 矩阵进行转置
tfidf_matrix_transposed = tfidf_matrix.T

# 对转置后的 TF-IDF 矩阵进行聚类
kmeans.fit(tfidf_matrix_transposed)

# 获取每个词汇所属的聚类标签
cluster_labels = kmeans.labels_

# 将聚类标签添加到词汇表中，以便分析
feature_names = vectorizer.get_feature_names_out()
feature_clusters = pd.DataFrame({'word': feature_names, 'cluster': cluster_labels})

# 查看每个聚类的大小
cluster_sizes = feature_clusters['cluster'].value_counts()

# 打印结果
print(cluster_sizes)


D:\anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


0    858
4    105
2     22
1      9
3      3
7      1
6      1
5      1
Name: cluster, dtype: int64


In [8]:
from collections import Counter

# 遍历每个聚类
for cluster_num in range(n_clusters):
    cluster_words = []
    
    # 获取当前聚类中的所有职位描述
    cluster_data = data[data['cluster'] == cluster_num]['description']
    
    # 遍历当前聚类中的每个职位描述
    for description in cluster_data:
        # 将职位描述拆分成单词
        words = description.split()
        cluster_words += words
    
    # 计算当前聚类中每个词汇的出现频率
    word_counts = Counter(cluster_words)
    
    # 打印当前聚类中最常见的词汇
    print(f"Cluster {cluster_num} most common words:")
    print(word_counts.most_common(10))
    print("\n")


Cluster 0 most common words:
[('and', 129), ('the', 76), ('of', 70), ('1.', 64), ('to', 48), ('岗位职责：1.', 42), ('in', 41), ('/', 38), ('with', 38), ('2.', 35)]


Cluster 1 most common words:
[('and', 1031), ('the', 422), ('to', 422), ('of', 389), ('in', 385), ('with', 302), ('a', 280), ('for', 205), ('?', 159), ('software', 136)]


Cluster 2 most common words:
[('1', 11), ('1.', 8), ('计算机、软件技术、信息工程等相关专业。2.', 7), ('2.', 6), ('2', 6), ('3.', 5), ('熟悉Oracle、MySQL、visual', 5), ('FoxPro等数据库开发，熟练使用SQL语言。4.', 5), ('4.', 4), ('熟练掌握计算机基本操作（Word、Excel、PPT），能够制作授课课件。3.', 4)]


Cluster 3 most common words:
[('1.', 23), ('3.', 20), ('4.', 18), ('岗位职责：1.', 17), ('2.', 16), ('/', 14), (',', 10), ('或', 8), ('5.', 8), ('-', 8)]


Cluster 4 most common words:
[('/', 64), ('岗位职责：1.', 63), ('1.', 60), ('2.', 42), ('and', 37), ('C', 32), ('3.', 31), ('on', 28), ('4.', 24), ('Linux', 24)]


Cluster 5 most common words:
[('?', 56), ('岗位职责：1.', 19), ('5.', 13), ('2.', 13), ('岗位职责：', 12), ('1、', 11), ('1.', 11)

In [9]:
# 为了简化，这里我们仅展示如何打印每个聚类中最常见的词汇
from collections import Counter

for cluster_num in range(n_clusters):
    cluster_words = []
    cluster_data = data[data['cluster'] == cluster_num]['description']
    for description in cluster_data:
        words = description.split()
        cluster_words += words
    most_common_words = Counter(cluster_words).most_common(10)
    print(f"Cluster {cluster_num} most common words:")
    print(most_common_words)
    print("\n")


Cluster 0 most common words:
[('and', 129), ('the', 76), ('of', 70), ('1.', 64), ('to', 48), ('岗位职责：1.', 42), ('in', 41), ('/', 38), ('with', 38), ('2.', 35)]


Cluster 1 most common words:
[('and', 1031), ('the', 422), ('to', 422), ('of', 389), ('in', 385), ('with', 302), ('a', 280), ('for', 205), ('?', 159), ('software', 136)]


Cluster 2 most common words:
[('1', 11), ('1.', 8), ('计算机、软件技术、信息工程等相关专业。2.', 7), ('2.', 6), ('2', 6), ('3.', 5), ('熟悉Oracle、MySQL、visual', 5), ('FoxPro等数据库开发，熟练使用SQL语言。4.', 5), ('4.', 4), ('熟练掌握计算机基本操作（Word、Excel、PPT），能够制作授课课件。3.', 4)]


Cluster 3 most common words:
[('1.', 23), ('3.', 20), ('4.', 18), ('岗位职责：1.', 17), ('2.', 16), ('/', 14), (',', 10), ('或', 8), ('5.', 8), ('-', 8)]


Cluster 4 most common words:
[('/', 64), ('岗位职责：1.', 63), ('1.', 60), ('2.', 42), ('and', 37), ('C', 32), ('3.', 31), ('on', 28), ('4.', 24), ('Linux', 24)]


Cluster 5 most common words:
[('?', 56), ('岗位职责：1.', 19), ('5.', 13), ('2.', 13), ('岗位职责：', 12), ('1、', 11), ('1.', 11)

In [10]:
cluster_num = 6
cluster_words = []
cluster_data = data[data['cluster'] == cluster_num]['description']
for description in cluster_data:
    words = description.split()
    cluster_words += words
most_common_words = Counter(cluster_words).most_common(50)
print(f"Cluster {cluster_num} most common words:")
print(most_common_words)
print("\n")

Cluster 6 most common words:
[('/', 73), ('Web', 68), ('1.', 54), ('+', 47), ('熟悉', 38), ('2.', 34), ('3.', 32), ('JavaScript', 31), ('React', 28), ('Vue', 26), ('-', 25), ('4.', 24), ('CSS', 22), ('5.', 21), ('UI', 19), ('岗位职责：1.', 19), ('Node.js', 18), ('JS', 17), ('熟练掌握', 16), ('vue', 15), ('HTTP', 15), ('熟练使用', 14), ('Node', 14), ('web', 13), ('HTML', 13), ('ES6', 13), ('或', 13), ('6.', 13), ('Webpack', 13), ('W3C', 12), ('等', 12), ('和', 12), ('react', 12), ('任职要求：', 10), ('、', 10), ('岗位职责：', 10), ('Angular', 10), ('精通', 10), ('js', 9), ('任职资格：', 9), ('3', 9), ('TypeScript', 8), (',', 8), ('岗位职责：1、', 7), ('负责前端界面的前端构建，各类交互设计与实现', 7), ('nodejs', 7), ('CSS3,', 7), ('1、', 7), ('双休', 7), ('及', 7)]




In [9]:
from collections import Counter
import jieba
# 初始化一个计数器
skills_counter = Counter()

# 假设data中有一列叫'description'，包含职位描述
for description in data['description']:
    # 使用jieba进行中文分词
    words = jieba.cut(description)
    # 更新计数器
    skills_counter.update(words)

# 假设技能词汇长度至少为2个字符，且至少出现了5次
skills_keywords = [skill for skill, count in skills_counter.items() if len(skill) > 2 and count >= 100]

print(skills_keywords)

['HTML5', 'JavaScript', 'CSS3', 'uni', 'app', '模块化', 'Vue', '数据分析', '系统维护', '表达能力', 'PPT', 'CAD', '数据库', '网络设备', '责任心', '实习生', 'Web', '程序开发', 'vue', 'JAVA', '全日制', '以上学历', '毕业生', 'Python', 'VUE', '有限公司', '互联网', '解决方案', '信息技术', '计算机', '软件开发', '设计方案', 'web', 'APP', 'css', 'javascript', 'react', '福利待遇', '节假日', '不定期', 'HTML', 'CSS', '逻辑思维', '实践经验', '岗位职责', '前端开发', 'Javascript', '理工科', '上进心', 'html', '工程师', '高质量', '设计师', '兼容性', '浏览器', '计算机专业', 'html5', 'webpack', '吃苦耐劳', '软件系统', '计算机软件', 'git', '功能模块', '产品开发', '熟练掌握', 'Ajax', 'jQuery', '兼容性问题', '程序设计', '测试计划', '项目经理', '软件工程', '通信工程', '专业本科', 'React', '责任感', '主动性', '基础知识', '应届生', '开发人员', '自动化', 'Angular', '解决问题', '计算机相关', 'JQuery', '人工智能', '认真负责', '管理系统', 'ES6', 'Node', '开发技术', 'WEB', '稳定性', '大学专科', '积极主动', '开发者', 'Git', '开发工具', 'SQL', 'Spring', '设计模式', '软件测试', '测试报告', 'HTTP', '前沿技术', '负责人', 'uniapp', '单元测试', 'W3C', '团队精神', '软硬件', '防火墙', '网络安全', '标准化', 'design', '试用期', '年终奖', '性格开朗', '上班时间', 'PHP', '管理工作', '电路设计', 'BIM', '上级领导', '专业知识', '信息化

In [12]:
import re

# 技能等级关键词和技能个人素质列表
level_keywords = {
    "精通": 4,
    "掌握": 3,
    "熟悉": 2,
    "了解": 1,
}
education_keywords = {
    "博士": 4,
    "硕士": 3,
    "本科": 2,
    "大专": 1, 
}

# 定义技能和个人素质列表
skills_list = [
    "HTML5", "JavaScript", "CSS3", "Vue", "Python", "JAVA", "React", "PHP", "C++", "C语言",
    "uni-app", "jQuery", "Angular", "Bootstrap", "Node.js", "Express", "Vue.js",
    "Git", "Webpack", "MySQL", "Oracle", "SQL Server", "Linux", "Windows", "Android",
     "CAD绘图", "系统集成", "电路设计", "BIM技术", "ERP系统","控制系统", "电子信息", "通信协议", "智能化", "机器人技术", "控制算法",
    "网络设备", "网络安全", "防火墙", "TCP/IP", "OSPF", "BGP", "VPN", "交换机", "路由器", "网络系统", "网络通信",
    "软件测试", "测试计划", "测试报告", "单元测试", "功能测试", "测试用例", "软件工程", "设计模式", "架构设计", "面向对象设计", "软件开发", "系统维护", "项目管理", "API开发",
    "数据分析", "数据结构", "数据库管理", "信息化", "信息安全", "信息系统", "数据中心", "图像处理", "音视频处理"
]

qualities_list = [
    "表达能力", "PPT制作", "售后服务", "技术支持", "方案设计", 
    "责任心", "上进心", "逻辑思维", "实践经验", "团队精神", "吃苦耐劳", "认真负责", "解决问题", "主动性","积极性", "沟通","语言表达"
]

def extract_and_rate_skills_and_qualities(description, skills_list, qualities_list):
    skills_ratings = []
    qualities_count = 0
#     for skill in skills_list:
#         for level, rating in level_keywords.items():
#             pattern = re.compile(f"{level}.*{skill}|{skill}.*{level}", re.IGNORECASE)
#             if pattern.search(description):
#                 skills_ratings.append((skill, rating))
#                 break
                
    for skill in skills_list:
        for level, rating in level_keywords.items():
            pattern = re.compile(f"{level}.*{skill}|{skill}.*{level}", re.IGNORECASE)
            if pattern.search(row["description"]):
                skills_ratings.append((skill, rating))
                break
        
        # 保留等级最高的技能
        if skills_ratings:
            max_rating = max(skills_ratings, key=lambda x: x[1])
            triples.append((row["title"], "技能要求", max_rating[0]))
    
    # 移除已有的相同技能要求，只保留等级最高的
    while True:
        if triples[-1][1] == "技能要求" and triples[-1][2] in skills_list:
            triples.pop()
        else:
            break
    for quality in qualities_list:
        if quality in description:
            qualities_count += 1
    return skills_ratings, qualities_count

# 应用抽取逻辑并构建三元组
triples = []
for _, row in data.iterrows():
    triples.append((row["company"], "职位", row["title"]))
    triples.append((row["title"], "薪资", row["salary"]))
    # 使用education_keywords转换教育要求
    education_score = education_keywords.get(row["education"], 0)  # 默认值为0，如果没有找到匹配项
    triples.append((row["title"], "教育要求", education_score))
    triples.append((row["title"], "招聘经理", row["hiring_manager"]))
    triples.append((row["title"], "活跃时间", row["last_active"]))
    triples.append((row["title"], "工作地点", row["address"]))
    triples.append((row["title"], "链接", row["link"]))
    
    skills_ratings, qualities_count = extract_and_rate_skills_and_qualities(row['description'], skills_list, qualities_list)
    
#     for skill, rating in skills_ratings:
#         triples.append((row['title'], skill, rating))  # 注意这里将技能等级也加入到了三元组中
    for skill, rating in skills_ratings:
        triples.append((row['title'], "技能要求", {"技能": skill, "等级": rating}))
    if qualities_count > 0:
        triples.append((row['title'], "个人素质", qualities_count))

# 展示部分三元组结果
triples[1100:1120]


[('初级前端工程师',
  '链接',
  'https://www.zhipin.com/job_detail/5db263093285404f0nR82Ni_EVE~.html?lid=6GPZx3II9Bv.search.135&securityId=QglvEd8kTjKmS-O1U0jpRjzSAOR1e4Ix0XS-neXo-3LuNjgjAfPVrjXnGkVM7rHdKwlOl6LSNErnqJvssNlEsu0cXG0CZqhkBWRfmVujnNMFIQ7khZuOx_dFuA9LAMMPAh7iyEcHiQ%7E%7E&sessionId='),
 ('初级前端工程师', '技能要求', {'技能': 'C++', '等级': 3}),
 ('初级前端工程师', '个人素质', 3),
 ('西安平衡语言科技有限公司', '职位', 'WEB前端开发实习'),
 ('WEB前端开发实习', '薪资', '1-2K'),
 ('WEB前端开发实习', '教育要求', 1),
 ('WEB前端开发实习', '招聘经理', '宋可'),
 ('WEB前端开发实习', '活跃时间', '半年前活跃'),
 ('WEB前端开发实习', '工作地点', '西安未央区阳光北郡4-3207'),
 ('WEB前端开发实习',
  '链接',
  'https://www.zhipin.com/job_detail/3e7802cdd687c75a1n1-2ti4ElJZ.html?lid=6GPZx3II9Bv.search.136&securityId=SlDBvIjKL5_K3-B1I-u9JyToKMn7yTQ5QJ-ofLBPrEvZLiBC4LcfUYcAOn_dipJU2cSdVUfvTFdoXSBKzQpUI_D1tMexDuQDXD5XPbMwD-MFdmc-5g5JQhLgM2q1EFeyPbFqNojdC2I~&sessionId='),
 ('WEB前端开发实习', '个人素质', 1),
 ('山西润源项目管理咨询有限公司', '职位', '一级造价工程师'),
 ('一级造价工程师', '薪资', '1-2K'),
 ('一级造价工程师', '教育要求', 1),
 ('一级造价工程师', '招聘经理', '梁昕'),
 ('一级造

In [13]:
triples[:20]

[('广东倾云科技有限公司', '职位', '【初级】web前端开发工程师'),
 ('【初级】web前端开发工程师', '薪资', '2-4K·13薪'),
 ('【初级】web前端开发工程师', '教育要求', 1),
 ('【初级】web前端开发工程师', '招聘经理', '梁先生'),
 ('【初级】web前端开发工程师', '活跃时间', '2周内活跃'),
 ('【初级】web前端开发工程师', '工作地点', '中山广东倾云科技有限公司一层'),
 ('【初级】web前端开发工程师',
  '链接',
  'https://www.zhipin.com/job_detail/2c08c111e75221011n1539S8EVVQ.html?lid=6GPZx3II9Bv.search.31&securityId=f1xZgMo04Kywv-u1CqKcTn5qoCHek_nDfH_HMLOZDR2U7UD6hrtSKuU33BsHCW7ByiFvb2dqc6C5mDevs78kfFXp601IBEUpA0Rr43wDm5o5vDLg5UeiSXzoNHZYpCYCdkPCHw5liZ3l&sessionId='),
 ('【初级】web前端开发工程师', '技能要求', {'技能': 'HTML5', '等级': 2}),
 ('【初级】web前端开发工程师', '技能要求', {'技能': 'JavaScript', '等级': 2}),
 ('【初级】web前端开发工程师', '技能要求', {'技能': 'CSS3', '等级': 2}),
 ('【初级】web前端开发工程师', '技能要求', {'技能': 'Vue', '等级': 2}),
 ('【初级】web前端开发工程师', '技能要求', {'技能': 'JAVA', '等级': 2}),
 ('【初级】web前端开发工程师', '技能要求', {'技能': 'C++', '等级': 2}),
 ('【初级】web前端开发工程师', '技能要求', {'技能': 'uni-app', '等级': 2}),
 ('火眼科技（天津）有限公司', '职位', 'IT运维工程师'),
 ('IT运维工程师', '薪资', '2-4K'),
 ('IT运维工程师', '教育要求', 1),
 

In [18]:
from py2neo import Graph, Node, Relationship

# Neo4j 连接配置
uri = "bolt://localhost:7687"  # 修改为您的 Neo4j 实例地址
username = "neo4j"  # 修改为您的用户名
password = "Lmq141592"  # 修改为您的密码

# 初始化 Neo4j 连接
graph = Graph(uri, auth=(username, password))

# def create_node_and_relationships(triples):
#     for subject, predicate, object in triples:
#         # 初始化标签
#         subject_label, object_label = "Entity", "Entity"
        
#         # 定义主语和宾语的标签
#         if predicate == "提供职位":
#             subject_label, object_label = "Company", "Position"
#         elif predicate in ["要求技能", "属于行业", "掌握技能", "拥有教育背景"]:
#             subject_label = "Position"
#             object_label = "Skill" if predicate in ["要求技能", "掌握技能"] else "Industry" if predicate == "属于行业" else "Education"
        
#         # 创建或获取主语节点
#         subject_node = graph.nodes.match(subject_label, name=subject).first()
#         if not subject_node:
#             subject_node = Node(subject_label, name=subject)
#             graph.create(subject_node)
        
#         # 特殊处理技能等级信息
#         if isinstance(object, dict) and "技能" in object and "等级" in object:
#             # 从字典中提取技能名称和等级
#             skill_name = object["技能"]
#             skill_level = object["等级"]
            
#             # 创建或获取技能节点
#             skill_node = graph.nodes.match("Skill", name=skill_name).first()
#             if not skill_node:
#                 skill_node = Node("Skill", name=skill_name)
#                 graph.create(skill_node)
            
#             # 创建包含等级信息的关系
#             relationship = Relationship(subject_node, predicate, skill_node)
#             relationship["等级"] = skill_level  # 将等级作为关系的属性
#             graph.create(relationship)
#         else:
#             # 对于其他情况，创建或获取宾语节点
#             object_node = graph.nodes.match(object_label, name=object).first()
#             if not object_node:
#                 object_node = Node(object_label, name=object)
#                 graph.create(object_node)
            
#             # 创建关系
#             relationship = Relationship(subject_node, predicate, object_node)
#             graph.create(relationship)

            
def create_node_and_relationships(triples):
    # 存储已经创建的节点以避免重复创建
    created_nodes = {
        "Company": {}, "Position": {}, "Skill": {}, "Education": {},
        "Salary": {}, "Quality": {}, "ActiveTime": {}
    }

    for subject, predicate, object, *attributes in triples:
        if predicate == "职位":
            # 处理公司和职位的关系
            company_node = created_nodes["Company"].setdefault(subject, Node("Company", name=subject))
            graph.merge(company_node, "Company", "name")

            position_node = Node("Position", name=object, company=subject)
            graph.merge(position_node, "Position", "name")
            created_nodes["Position"][object] = position_node

            relationship = Relationship(company_node, "OFFERS", position_node)
            graph.merge(relationship)

        elif predicate == "技能要求":
            # 特殊处理技能要求，技能等级作为关系属性
            position_node = created_nodes["Position"][subject]
            skill_node = created_nodes["Skill"].setdefault(object['技能'], Node("Skill", name=object['技能']))
            graph.merge(skill_node, "Skill", "name")

            # 建立关系，并设置技能等级属性
            rel = Relationship(position_node, "REQUIRES_SKILL", skill_node, level=object['等级'])
            graph.create(rel)

        elif predicate in ["薪资", "教育要求", "个人素质", "活跃时间"]:
            # 创建独立节点并建立关系
            position_node = created_nodes["Position"][subject]
            # 将中文键映射到英文键
            predicate_mapping = {"薪资": "Salary", "教育要求": "Education", "个人素质": "Quality", "活跃时间": "ActiveTime"}
            english_predicate = predicate_mapping[predicate]

            attribute_node = created_nodes[english_predicate].setdefault(object, Node(english_predicate, name=object))
            graph.merge(attribute_node, english_predicate, "name")

            relationship = Relationship(position_node, english_predicate.upper(), attribute_node)
            graph.create(relationship)

        elif predicate == "链接":
            # 链接作为职位的属性
            position_node = created_nodes["Position"][subject]
            position_node['链接'] = object
            graph.push(position_node)

# 导入三元组数据到 Neo4j
create_node_and_relationships(triples)


In [11]:
# from py2neo import Graph, Node, Relationship

# # Neo4j 连接配置
# uri = "bolt://localhost:7687"  # 修改为您的 Neo4j 实例地址
# username = "neo4j"  # 修改为您的用户名
# password = "Lmq141592"  # 修改为您的密码

# # 初始化 Neo4j 连接
# graph = Graph(uri, auth=(username, password))

# # 定义一个函数来创建节点和关系
# def create_node_and_relationships(triples):
#     for subject, predicate, object, attributes in triples:
#         # 根据predicate确定节点类型
#         subject_label = "Company" if predicate in ["提供职位", "招聘经理"] else "Position"
#         object_label = "Position" if predicate == "提供职位" else "Skill"

#         # 创建或获取主语节点
#         subject_node = graph.nodes.match(subject_label, name=subject).first()
#         if not subject_node:
#             subject_node = Node(subject_label, name=subject)
#             graph.create(subject_node)

#         # 创建或获取宾语节点，为每个节点加入额外属性
#         object_node = graph.nodes.match(object_label, name=object).first()
#         if not object_node:
#             object_node = Node(object_label, name=object, **attributes)
#             graph.create(object_node)

#         # 创建关系，对于技能等级，将其作为关系的属性
#         if predicate == "要求技能":
#             relationship = Relationship(subject_node, predicate, object_node, level=attributes.get("level"))
#         else:
#             relationship = Relationship(subject_node, predicate, object_node)
#         graph.create(relationship)

# # 导入三元组数据到 Neo4j
# create_node_and_relationships(triples)


In [1]:
from py2neo import Graph

# Neo4j 连接配置
uri = "bolt://localhost:7687"  # 修改为您的 Neo4j 实例地址
username = "neo4j"              # 修改为您的用户名
password = "Lmq141592"          # 修改为您的密码

# 初始化 Neo4j 连接
graph = Graph(uri, auth=(username, password))






In [2]:
query = """
MATCH (p:Position)-[r:REQUIRES_SKILL]->(s:Skill)
WITH CASE WHEN exists(r.level) THEN r.level ELSE 'None' END AS skillLevel, COUNT(*) AS count
RETURN skillLevel, count ORDER BY skillLevel
"""

results = graph.run(query)
print(results)

 skillLevel | count 
------------|-------
          1 |   307 
          2 |  3141 
          3 |  3265 



In [4]:
query = """
MATCH ()-[r:REQUIRES_SKILL]->() WHERE NOT exists(r.level) RETURN COUNT(*);
"""

results = graph.run(query)
print(results)

 COUNT(*) 
----------
        0 

